In [1]:
import pandas as pd
import numpy as np

import fuzzywuzzy
from fuzzywuzzy import process
import chardet


In [5]:
suicide_attacks = pd.read_csv("E:/Beuth_Applications/Data/CS/Challenge/PakistanSuicideAttacks Ver 11 (30-November-2017).csv", encoding='Windows-1252')

In [6]:
suicide_attacks.head()

,S#,Date,Islamic Date,Blast Day Type,Holiday Type,Time,City,Latitude,Longitude,Province,...,Targeted Sect if any,Killed Min,Killed Max,Injured Min,Injured Max,No. of Suicide Blasts,Explosive Weight (max),Hospital Names,Temperature(C),Temperature(F)
0,1,Sunday-November 19-1995,25 Jumaada al-THaany 1416 A.H,Holiday,Weekend,NaN,Islamabad,33.7180,73.0718,Capital,...,None,14.0,15.0,NaN,60,2.0,NaN,NaN,15.835,60.503
1,2,Monday-November 6-2000,10 SHa`baan 1421 A.H,Working Day,NaN,NaN,Karachi,24.9918,66.9911,Sindh,...,None,NaN,3.0,NaN,3,1.0,NaN,NaN,23.770,74.786
2,3,Wednesday-May 8-2002,25 safar 1423 A.H,Working Day,NaN,7:45 AM,Karachi,24.9918,66.9911,Sindh,...,Christian,13.0,15.0,20.0,40,1.0,2.5 Kg,1.Jinnah Postgraduate Medical Center 2. Civil ...,31.460,88.628
3,4,Friday-June 14-2002,3 Raby` al-THaany 1423 A.H,Working Day,NaN,11:10:00 AM,Karachi,24.9918,66.9911,Sindh,...,Christian,NaN,12.0,NaN,51,1.0,NaN,NaN,31.430,88.574
4,5,Friday-July 4-2003,4 Jumaada al-awal 1424 A.H,Working Day,NaN,NaN,Quetta,30.2095,67.0182,Baluchistan,...,Shiite,44.0,47.0,NaN,65,1.0,NaN,1.CMH Quetta \n2.Civil Hospital 3. Boland Medi...,33.120,91.616


In [7]:
suicide_attacks['Province'] = suicide_attacks['Province'].str.lower()

suicide_attacks['Province'] = suicide_attacks['Province'].str.strip()

In [9]:
cities = suicide_attacks['City'].unique()
matches = fuzzywuzzy.process.extract("kuram agency", cities, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
matches

[('Kuram Agency ', 100),
 ('KURRAM AGENCY', 96),
 ('Bajaur Agency', 72),
 ('Khyber Agency', 72),
 ('Khyber Agency ', 72),
 ('Orakzai Agency', 69),
 ('Mohmand agency', 62),
 ('Mohmand Agency', 62),
 ('Mohmand Agency ', 62),
 ('Mosal Kor, Mohmand Agency', 61)]

In [10]:
def replace_matches_in_column(df, column, string_to_match, min_ratio = 90):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")

In [11]:
replace_matches_in_column(df=suicide_attacks, column='City', string_to_match="kuram agency")

All done!


In [12]:
cities = suicide_attacks['City'].unique()

In [13]:
cities

array(['Islamabad', 'Karachi', 'Karachi ', 'Quetta', 'Rawalpindi',
       'North waziristan', 'Kohat', 'Attock ', 'Sialkot', 'Lahore ',
       'Swat', 'Hangu', 'Bannu', 'North Waziristan', 'Lasbela',
       'Malakand ', 'Peshawar', 'Peshawar ', 'D.I Khan ', 'Lakki Marwat',
       'Tank', 'Islamabad ', 'Quetta ', 'Gujrat', 'Charsadda', 'Tank ',
       'Swat ', 'D.I Khan', 'Kohat ', 'kuram agency', 'Shangla ',
       'Bajaur Agency', 'South Waziristan', 'Haripur', 'bannu',
       'karachi ', 'Sargodha', 'Nowshehra', 'Charsadda ',
       'Mohmand agency', 'Dara Adam Khel', 'Khyber Agency', 'Mardan',
       'Bhakkar ', 'Orakzai Agency', 'Mohmand Agency', 'Buner',
       'D.G Khan ', 'Pishin', 'Chakwal ', 'Lahore', 'Upper Dir',
       'Muzaffarabad', 'swat', 'Totalai', 'Multan', 'D.G Khan',
       'Lower Dir', 'Lakki marwat', 'Sudhanoti', 'Khyber Agency ',
       'peshawar', 'Poonch', 'Mansehra', 'Karak', 'Mohmand Agency ',
       'South waziristan', 'lakki marwat', 'Swabi ', 'Dara Adam khe